In [1]:
import torch
from torch import nn
from torch.autograd import Variable

In [2]:
from datasets import load_dataset


dataset = load_dataset('sms_spam')


2022-05-31 03:50:36.783935: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-31 03:50:36.783956: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Reusing dataset sms_spam (/home/max/.cache/huggingface/datasets/sms_spam/plain_text/1.0.0/53f051d3b5f62d99d61792c91acefe4f1577ad3e4c216fb0ad39e30b9f20019c)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
dataset['train']

Dataset({
    features: ['sms', 'label'],
    num_rows: 5574
})

In [4]:
from torchtext import vocab

PAD_WORD = '<pad>'

def get_vocab(X):
    return vocab.build_vocab_from_iterator([sentence.split() for sentence in X], specials=["<unk>", "<pad>"])

def pad(seq, size):
    if len(seq) < size:
        seq = seq + [PAD_WORD] * (size - len(seq))
    return seq

def to_indices(vocab, words):
    stoi = vocab.get_stoi()
    return [stoi[w] for w in words]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sms', 'label'],
        num_rows: 5574
    })
})

In [6]:
cut = 1000

from random import sample

newA = [(a, b) for a, b in zip(dataset['train']['sms'], dataset['train']['label'])]

newA = sample(newA, cut)


newB = [(a, b) for a, b in zip(dataset['train']['sms'], dataset['train']['label'])]

newB = sample(newB, cut)


X_train = []
Y_train = []
for a, b in newA:
    X_train.append(a)
    Y_train.append(b)

X_test = []
Y_test = []
for a, b in newB:
    X_test.append(a)
    Y_test.append(b)
    

In [7]:
maxlen = 0
for str1 in X_train:
    maxlen = max(maxlen, len(str1))

for str1 in X_test:
    maxlen = max(maxlen, len(str1))
    
maxlen

589

In [8]:
vocab1 = get_vocab(X_train + X_test)
vocab1

Vocab()

In [9]:
vocab_size = len(vocab1)
vocab_size

7626

In [10]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert = BertModel.from_pretrained('bert-base-uncased')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
import torch.nn as nn

In [12]:
from torch.utils.data import TensorDataset
import enum

MAX_SEQ = 32

class Model(enum.Enum):
    BERT = 3
    DISTIL_BILSTM = 2
    CLASSIC_BILSTM = 1

def bert_token(X, y, tokenizer):
    lines = [" ".join(s) for s in X]
    masks = [[int(word != PAD_WORD) for word in sentence] for sentence in X]
    inds = torch.tensor([tokenizer.encode(line.split(), add_special_tokens=False) for line in lines])
    masks = torch.tensor(masks, dtype=torch.int8)
    torch_y = torch.tensor(y, dtype=torch.float)
    return TensorDataset(inds, torch_y, masks)

def bilstm_token(X, y, vocab, teacher_output, model):
    torch_x = torch.tensor(X, dtype=torch.long)
    torch_y = torch.tensor(y, dtype=torch.float)

    if model == Model.DISTIL_BILSTM:
        return TensorDataset(torch_x, torch_y, teacher_output)
    else:
        return TensorDataset(torch_x, torch_y)

def tokenize(X, y, model = Model.CLASSIC_BILSTM, vocab=None, teacher_output = None, tokenizer = None):    
    X_padded = [pad(s, MAX_SEQ) for s in [t.split()[:MAX_SEQ] for t in X]]
    
    if model == Model.BERT:
        return bert_token(X_padded, y, tokenizer)
    else:
        return bilstm_token([to_indices(vocab, s) for s in X_padded], y, vocab, teacher_output, model)

In [13]:

import torch.nn.functional as F

class TeacherModel(nn.Module):
    def init(self, teacher):
        super(TeacherModel, self).init()
        self.teacher = teacher
        self.fc1 = nn.Linear(self.teacher.config.hidden_size, 2)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, inp):
        inds, labels, masks = inp
        labels = torch.tensor(labels, dtype=torch.long)
        output = self.teacher(inds, attention_mask=masks)[0][:, 0, :]
        pred = self.fc1(output)
        loss = self.loss(pred, labels)
        return loss, pred


In [14]:
bert = TeacherModel(bert)

In [15]:
bert_dataset_train = tokenize(X_train, Y_train, tokenizer=tokenizer, model=Model.BERT)

In [16]:
bert_dataset_test = tokenize(X_test, Y_test, tokenizer=tokenizer, model=Model.BERT)

In [17]:
from torch.utils.data import DataLoader
from torch.optim import Adam
from tqdm import tqdm

device = torch.device("cpu")
batch_size = 8

def train(model, dataset, epochs=5):
    dataloader = DataLoader(dataset, batch_size, shuffle=True)
    optimizer = Adam(model.parameters())
    model.train()
    for e in range(epochs):
        print("epoch", e)
        losses = 0
        count = 0
        for info in tqdm(dataloader):
            loss, _ = model(info)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses += loss
            count += 1
        losses /= count
        print(f'Loss: {losses.item()}')

In [62]:
train(bert, bert_dataset_train, bert_dataset_test, epochs=5)

  0%|                                                                                                                                                                                                               | 0/125 [00:00<?, ?it/s]/tmp/ipykernel_70710/289778998.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 125/125 [02:41<00:00,  1.30s/it]


Epoch 0 	| Loss: 0.32838818430900574


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 125/125 [03:02<00:00,  1.46s/it]


Epoch 1 	| Loss: 0.27588585019111633


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 125/125 [02:29<00:00,  1.20s/it]


...

Epoch 3 	| Loss: 0.23004834353923798


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 125/125 [02:18<00:00,  1.11s/it]


Epoch 4 	| Loss: 0.21688580513000488


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 125/125 [00:48<00:00,  2.60it/s]


Accuracy:0.8899999856948853


In [18]:

def get_output(modell, dataset):
    dataloader = DataLoader(dataset, batch_size)
    output = []
    with torch.no_grad():
        for info in tqdm(dataloader):
            loss, result = modell(info)
            output.append(result)
        output = torch.cat(output)
    return output




In [19]:
import pickle

loaded_bert_model = pickle.load(open("bert.model", 'rb'))

In [20]:

bert_output_train = get_output(loaded_bert_model, bert_dataset_train)
bert_output_test = get_output(loaded_bert_model, bert_dataset_test)

  0%|                                                                                                                                                                                                               | 0/125 [00:00<?, ?it/s]/tmp/ipykernel_86841/289778998.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 125/125 [00:49<00:00,  2.52it/s]


In [21]:
import sklearn.metrics

sklearn.metrics.accuracy_score(Y_test, torch.argmax(bert_output_test, axis=1))

0.909

In [22]:

distil_dataset_train = tokenize(X_train, Y_train, teacher_output=bert_output_train, vocab=vocab1, model=Model.DISTIL_BILSTM)


In [23]:

distil_dataset_test = tokenize(X_test, Y_test, teacher_output=bert_output_test, vocab=vocab1, model=Model.DISTIL_BILSTM)



In [32]:

class DistilModel(nn.Module):
    def __init__(self, student, alpha=0.75):
        super(DistilModel, self).__init__()
        self.student = student
        self.alpha = alpha

    def loss(self, real_prediction, real_output, teacher_prediction, teacher_output):
        bce = nn.CrossEntropyLoss()
        mse = nn.MSELoss()
        prediction_loss = bce(real_prediction, torch.tensor(real_output, dtype=torch.long))
        teacher_loss = mse(teacher_prediction, teacher_output)
        return self.alpha * prediction_loss + (1 - self.alpha) * teacher_loss

    def forward(self, inp):
        inds, labels, teacher_output = inp
        labels = torch.tensor(labels, dtype=torch.long)
        label_prediction = self.student(inds)
        loss = self.loss(label_prediction, labels, label_prediction, teacher_output)
        return loss, label_prediction

class SimpleModel(nn.Module):
    def __init__(self, bilstm):
        super(SimpleModel, self).__init__()
        self.bilstm = bilstm
        self.loss = nn.CrossEntropyLoss()

    def forward(self, inp):
        inds, labels = inp
        labels = torch.tensor(labels, dtype=torch.long)
        prediction = self.bilstm(inds)
        loss = self.loss(prediction, labels)
        return loss, prediction
    

In [33]:
class SimpleLSTM(nn.Module):

    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, n_layers,
                 bidirectional, dropout, device=None):
        super(SimpleLSTM, self).__init__()
        self.batch_size = batch_size
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, embedding_dim)

        self.rnn = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional,
                           dropout=dropout)

        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)


    def init_state(self, batch_size):
        return torch.zeros(2 * self.num_layers, batch_size, self.hidden_dim), \
               torch.zeros(2 * self.num_layers, batch_size, self.hidden_dim)

    def forward(self, text, text_lengths=None):
        x = self.embedding(text)
        x = torch.transpose(x, dim0=1, dim1=0)
        x, self.hidden = self.rnn(x)
        hidden, cell = self.hidden
        hidden = self.dropout(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1))
        x = self.fc(hidden)

        return x


In [34]:
distil_model = SimpleLSTM(
            input_dim=len(vocab1),
            embedding_dim=16,
            hidden_dim=16,
            output_dim=2,
            n_layers=1,
            bidirectional=True,
            dropout=0.5)

student = DistilModel(distil_model, alpha=0.5)


/home/max/.local/lib/python3.10/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [35]:
train(student, distil_dataset_train, epochs=10)

epoch 0


  0%|                                                                                                                                                                                                               | 0/125 [00:00<?, ?it/s]/tmp/ipykernel_86841/1172701668.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long)
/tmp/ipykernel_86841/1172701668.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  prediction_loss = bce(real_prediction, torch.tensor(real_output, dtype=torch.long))
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

Loss: 2.58984637260437
epoch 1


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 125/125 [00:01<00:00, 80.60it/s]


Loss: 1.455444574356079
epoch 2


...

Loss: 0.8994405269622803
epoch 8


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 125/125 [00:02<00:00, 59.32it/s]


Loss: 0.8417882323265076
epoch 9


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 125/125 [00:02<00:00, 57.15it/s]

Loss: 0.8652071356773376


In [36]:
sklearn.metrics.accuracy_score(Y_test, torch.argmax(get_output(student, distil_dataset_test), axis=1))

  0%|                                                                                                                                                                                                               | 0/125 [00:00<?, ?it/s]/tmp/ipykernel_86841/1172701668.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long)
/tmp/ipykernel_86841/1172701668.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  prediction_loss = bce(real_prediction, torch.tensor(real_output, dtype=torch.long))
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

0.896

In [37]:

base_model = SimpleLSTM(input_dim=vocab_size,
               embedding_dim=16,
               hidden_dim=16,
               output_dim=2,
               n_layers=1,
               bidirectional=True,
               dropout=0.8)

basic_model = SimpleModel(base_model)

/home/max/.local/lib/python3.10/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [38]:

basic_dataset_train = tokenize(X_train, Y_train, vocab=vocab1, model=Model.CLASSIC_BILSTM)


In [39]:

basic_dataset_test = tokenize(X_test, Y_test, vocab=vocab1, model=Model.CLASSIC_BILSTM)


In [40]:
train(basic_model, basic_dataset_train, epochs=10)

epoch 0


  0%|                                                                                                                                                                                                               | 0/125 [00:00<?, ?it/s]/tmp/ipykernel_86841/1172701668.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 125/125 [00:01<00:00, 64.59it/s]


Loss: 0.4701578915119171
epoch 1


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 125/125 [00:02<00:00, 62.12it/s]


Loss: 0.34898239374160767
epoch 2


...

Loss: 0.14020206034183502
epoch 8


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 125/125 [00:02<00:00, 59.96it/s]


Loss: 0.1163373813033104
epoch 9


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 125/125 [00:02<00:00, 56.31it/s]

Loss: 0.13328538835048676


In [41]:
sklearn.metrics.accuracy_score(Y_test, torch.argmax(get_output(basic_model, basic_dataset_test), axis=1))

  0%|                                                                                                                                                                                                               | 0/125 [00:00<?, ?it/s]/tmp/ipykernel_86841/1172701668.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long)
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 125/125 [00:00<00:00, 281.72it/s]


0.912